In [ ]:
"""
Production Readiness Audit for Agent Systems
7-Layer Framework Assessment Tool

Created by: Nachiketh
Website: https://community.nachiketh.in
Bootcamp: https://bootcamp.nachiketh.in

Instructions:
1. Run all cells in order
2. Answer the diagnostic questions honestly
3. Get your production readiness score
4. Review your gaps and next steps

This is a diagnostic tool, not a solution.
It shows you what's missing in your system.
"""

# ==============================================================================
# SETUP
# ==============================================================================

import json
from typing import Dict, List, Tuple
from dataclasses import dataclass
from enum import Enum

print("🎯 Production Readiness Audit for Agent Systems")
print("=" * 60)
print()

# ==============================================================================
# SCORING FRAMEWORK
# ==============================================================================

class ScoreLevel(Enum):
    YES = 10  # Fully implemented
    PARTIAL = 5  # Partially implemented
    NO = 0  # Not implemented

@dataclass
class Question:
    id: str
    layer: int
    text: str
    weight: int = 10  # All questions weighted equally

@dataclass
class AuditResult:
    total_score: float
    layer_scores: Dict[int, int]
    gaps: List[str]
    risk_assessment: str
    recommendations: List[str]

# ==============================================================================
# AUDIT QUESTIONS (7 Layers × 5 Questions = 35 Questions)
# ==============================================================================

QUESTIONS = [
    # LAYER 1: PROMPT ENGINEERING
    Question("L1Q1", 1, "Do you have versioned prompts with change tracking?"),
    Question("L1Q2", 1, "Do you have A/B testing infrastructure for prompts?"),
    Question("L1Q3", 1, "Do you have prompt templates for different scenarios?"),
    Question("L1Q4", 1, "Do you measure prompt effectiveness metrics?"),
    Question("L1Q5", 1, "Do you have fallback prompts for edge cases?"),

    # LAYER 2: TOOL SELECTION
    Question("L2Q1", 2, "Do you have analytics on tool usage patterns?"),
    Question("L2Q2", 2, "Do you have fallback tools when primary tools fail?"),
    Question("L2Q3", 2, "Do you validate tool outputs before using them?"),
    Question("L2Q4", 2, "Do you have timeout limits on tool execution?"),
    Question("L2Q5", 2, "Do you log all tool calls with parameters?"),

    # LAYER 3: RAG & MEMORY
    Question("L3Q1", 3, "Do you have metrics on retrieval quality?"),
    Question("L3Q2", 3, "Do you have a chunking strategy with overlap?"),
    Question("L3Q3", 3, "Do you handle context window overflow gracefully?"),
    Question("L3Q4", 3, "Do you have multiple retrieval strategies (hybrid search)?"),
    Question("L3Q5", 3, "Do you track which documents are most useful?"),

    # LAYER 4: AGENT REASONING
    Question("L4Q1", 4, "Do you log reasoning traces for debugging?"),
    Question("L4Q2", 4, "Do you have infinite loop detection?"),
    Question("L4Q3", 4, "Do you have max iteration limits?"),
    Question("L4Q4", 4, "Do you validate decisions before executing?"),
    Question("L4Q5", 4, "Do you have decision rollback mechanisms?"),

    # LAYER 5: ORCHESTRATION (CRITICAL - Most systems fail here)
    Question("L5Q1", 5, "Do you have retry logic with exponential backoff?"), # 10, 5, 0
    Question("L5Q2", 5, "Do you use idempotency keys on ALL external API calls?"),
    Question("L5Q3", 5, "Do you have circuit breakers for external services?"),
    Question("L5Q4", 5, "Do you have aggressive timeouts (2-5s) with retries?"),
    Question("L5Q5", 5, "Do you prevent duplicate actions (DB writes, API calls)?"),

    # LAYER 6: OBSERVABILITY (CRITICAL - Second most common gap)
    Question("L6Q1", 6, "Do you have structured logging (JSON format)?"),
    Question("L6Q2", 6, "Do you track cost per request?"),
    Question("L6Q3", 6, "Do you have distributed tracing with request IDs?"),
    Question("L6Q4", 6, "Do you have automated alerts for failures/anomalies?"),
    Question("L6Q5", 6, "Do you have dashboards showing system health?"),

    # LAYER 7: USER INTERFACE
    Question("L7Q1", 7, "Do you have API versioning?"),
    Question("L7Q2", 7, "Do you return proper error responses (not just 500)?"),
    Question("L7Q3", 7, "Do you have rate limiting per user/API key?"),
    Question("L7Q4", 7, "Do you have authentication and authorization?"),
    Question("L7Q5", 7, "Do you have input validation on all endpoints?"),
]

LAYER_NAMES = {
    1: "Prompt Engineering",
    2: "Tool Selection",
    3: "RAG & Memory",
    4: "Agent Reasoning",
    5: "Orchestration",
    6: "Observability",
    7: "User Interface"
}

# ==============================================================================
# INTERACTIVE AUDIT
# ==============================================================================

def run_audit() -> AuditResult:
    """Run the interactive audit"""

    print("\n" + "="*60)
    print("🔍 PRODUCTION READINESS AUDIT")
    print("="*60)
    print()
    print("Answer each question honestly:")
    print("  YES (10 pts) - Fully implemented")
    print("  PARTIAL (5 pts) - Partially implemented")
    print("  NO (0 pts) - Not implemented")
    print()

    answers = {}
    layer_scores = {i: 0 for i in range(1, 8)}

    current_layer = 0
    for q in QUESTIONS:
        # Print layer header when we start a new layer
        if q.layer != current_layer:
            print("\n" + "-"*60)
            print(f"LAYER {q.layer}: {LAYER_NAMES[q.layer].upper()}")
            print("-"*60 + "\n")
            current_layer = q.layer

        # Ask question
        while True:
            response = input(f"{q.id}: {q.text}\n  [Y]es / [P]artial / [N]o: ").strip().upper()

            if response in ['Y', 'YES']:
                answers[q.id] = ScoreLevel.YES.value
                layer_scores[q.layer] += ScoreLevel.YES.value
                break
            elif response in ['P', 'PARTIAL']:
                answers[q.id] = ScoreLevel.PARTIAL.value
                layer_scores[q.layer] += ScoreLevel.PARTIAL.value
                break
            elif response in ['N', 'NO']:
                answers[q.id] = ScoreLevel.NO.value
                layer_scores[q.layer] += ScoreLevel.NO.value
                break
            else:
                print("  ❌ Invalid input. Please enter Y, P, or N")

    # Calculate total score (0-350 points, convert to 0-100 scale)
    total_points = sum(layer_scores.values())
    total_score = (total_points / 350) * 100

    # Identify gaps
    gaps = identify_gaps(layer_scores, answers)

    # Risk assessment
    risk = assess_risk(total_score, layer_scores)

    # Generate recommendations
    recommendations = generate_recommendations(total_score, layer_scores, gaps)

    return AuditResult(
        total_score=total_score,
        layer_scores=layer_scores,
        gaps=gaps,
        risk_assessment=risk,
        recommendations=recommendations
    )

def identify_gaps(layer_scores: Dict[int, int], answers: Dict[str, int]) -> List[str]:
    """Identify critical gaps in the system"""
    gaps = []

    # Check each layer
    for layer, score in layer_scores.items():
        layer_name = LAYER_NAMES[layer]

        if score < 20:  # Less than 40% of max (50 points)
            gaps.append(f"CRITICAL: {layer_name} - Score {score}/50 - Severe gaps")
        elif score < 35:  # Less than 70% of max
            gaps.append(f"WARNING: {layer_name} - Score {score}/50 - Multiple gaps")

    # Special attention to Layers 5 and 6 (most critical)
    if layer_scores[5] < 40:
        gaps.append("🚨 URGENT: Layer 5 (Orchestration) is critically weak - HIGH RISK OF PRODUCTION FAILURES")

    if layer_scores[6] < 30:
        gaps.append("🚨 URGENT: Layer 6 (Observability) is critically weak - NO VISIBILITY WHEN THINGS FAIL")

    return gaps

def assess_risk(total_score: float, layer_scores: Dict[int, int]) -> str:
    """Assess production deployment risk"""

    # Critical layers
    layer5_score = layer_scores[5]
    layer6_score = layer_scores[6]

    if total_score >= 90:
        return "LOW RISK - Production-ready. Deploy with confidence."
    elif total_score >= 70:
        if layer5_score < 40 or layer6_score < 30:
            return "HIGH RISK - Core score is good but critical infrastructure gaps exist. Fix Layers 5-6 before scaling."
        else:
            return "MEDIUM RISK - Deploy with caution. Monitor closely and fix identified gaps."
    elif total_score >= 50:
        return "HIGH RISK - NOT production-ready. Major gaps will cause failures. DO NOT deploy until fixed."
    else:
        return "CRITICAL RISK - Demo/POC quality. WILL fail catastrophically. Rebuild with proper architecture."

def generate_recommendations(total_score: float, layer_scores: Dict[int, int], gaps: List[str]) -> List[str]:
    """Generate actionable recommendations"""
    recommendations = []

    if total_score >= 90:
        recommendations.append("✅ You're production-ready! Deploy and monitor for edge cases.")
        recommendations.append("💡 Focus on: Cost optimization, performance tuning, edge case handling")
        recommendations.append("🔗 Join community for ongoing patterns: https://community.nachiketh.in")

    elif total_score >= 70:
        recommendations.append("⚠️ Fix critical gaps before scaling:")

        if layer_scores[5] < 40:
            recommendations.append("  1. URGENT: Add retry logic, idempotency, circuit breakers (Layer 5)")
        if layer_scores[6] < 30:
            recommendations.append("  2. URGENT: Add structured logging, cost tracking, alerts (Layer 6)")

        recommendations.append("📅 Timeline: 1-2 weeks to fix gaps")
        recommendations.append("🔗 Implementation guides: https://community.nachiketh.in")
        recommendations.append("🎓 Learn all 7 layers: https://bootcamp.nachiketh.in")

    elif total_score >= 50:
        recommendations.append("🚨 DO NOT DEPLOY - Major production gaps detected")
        recommendations.append("Priority fixes:")
        recommendations.append("  1. Layer 5 (Orchestration) - Complete rebuild needed")
        recommendations.append("  2. Layer 6 (Observability) - Add basics: logging, metrics, alerts")
        recommendations.append("  3. Layer 7 (API Design) - Proper error handling, rate limiting")
        recommendations.append("📅 Timeline: 3-4 weeks to rebuild properly")
        recommendations.append("🎓 RECOMMENDED: Join Agentic AI Enterprise Bootcamp")
        recommendations.append("   → We build all 7 layers, production-grade, deployed")
        recommendations.append("   → Starts Feb 15: https://bootcamp.nachiketh.in")

    else:  # Below 50
        recommendations.append("🛑 STOP - System is not production-ready")
        recommendations.append("This is demo quality. You're missing fundamental infrastructure.")
        recommendations.append("Deploying this WILL cause catastrophic failures.")
        recommendations.append("")
        recommendations.append("Recommendation:")
        recommendations.append("Learn to build production agent systems properly.")
        recommendations.append("")
        recommendations.append("🎓 Agentic AI Enterprise Bootcamp:")
        recommendations.append("   → Build all 7 layers from scratch")
        recommendations.append("   → 7 production deployments")
        recommendations.append("   → Complete observability & cost optimization")
        recommendations.append("   → For Senior ML Engineers, Software Architects, Tech Leads")
        recommendations.append("   → Starts Feb 15: https://bootcamp.nachiketh.in")

    return recommendations

# ==============================================================================
# RESULTS DISPLAY
# ==============================================================================

def display_results(result: AuditResult):
    """Display audit results with formatting"""

    print("\n\n")
    print("="*60)
    print("📊 AUDIT RESULTS")
    print("="*60)
    print()

    # Overall score
    print(f"Overall Score: {result.total_score:.1f}/100")
    print()

    # Score interpretation
    if result.total_score >= 90:
        print("✅ Production-Ready")
    elif result.total_score >= 70:
        print("⚠️  Deploy with Caution")
    elif result.total_score >= 50:
        print("🚨 NOT Production-Ready")
    else:
        print("🛑 Demo/POC Quality")

    print()
    print("-"*60)
    print("📈 PER-LAYER SCORES")
    print("-"*60)
    print()

    # Layer scores with visual bars
    for layer in range(1, 8):
        score = result.layer_scores[layer]
        max_score = 50
        percentage = (score / max_score) * 100

        # Visual bar
        bar_length = int(percentage / 2)  # 50 chars max
        bar = "█" * bar_length + "░" * (50 - bar_length)

        # Color code
        if percentage >= 80:
            status = "✅"
        elif percentage >= 60:
            status = "⚠️ "
        else:
            status = "🚨"

        print(f"Layer {layer}: {LAYER_NAMES[layer]:<20} {status}")
        print(f"  Score: {score}/50 ({percentage:.0f}%)")
        print(f"  {bar}")
        print()

    # Critical gaps
    if result.gaps:
        print("-"*60)
        print("🔍 CRITICAL GAPS IDENTIFIED")
        print("-"*60)
        print()
        for gap in result.gaps:
            print(f"  {gap}")
        print()

    # Risk assessment
    print("-"*60)
    print("⚠️  RISK ASSESSMENT")
    print("-"*60)
    print()
    print(f"  {result.risk_assessment}")
    print()

    # Recommendations
    print("-"*60)
    print("💡 RECOMMENDATIONS")
    print("-"*60)
    print()
    for rec in result.recommendations:
        print(f"  {rec}")
    print()

    print("="*60)
    print()

# ==============================================================================
# MAIN EXECUTION
# ==============================================================================

print("""
This audit assesses your agent system across 7 layers:

Layer 1: Prompt Engineering (versioning, testing, templates)
Layer 2: Tool Selection (analytics, fallbacks, validation)
Layer 3: RAG & Memory (retrieval, chunking, overflow handling)
Layer 4: Agent Reasoning (tracing, loop detection, validation)
Layer 5: Orchestration (retry, idempotency, circuit breakers) ← CRITICAL
Layer 6: Observability (logging, metrics, tracing, alerts) ← CRITICAL
Layer 7: User Interface (versioning, errors, rate limiting, auth)

35 questions total. Takes 5-10 minutes.

Ready to start? Press Enter to begin...
""")
input()

# Run the audit
result = run_audit()

# Display results
display_results(result)

# Save results (optional)
save_choice = input("Would you like to save these results? [Y/N]: ").strip().upper()
if save_choice in ['Y', 'YES']:
    results_dict = {
        "total_score": result.total_score,
        "layer_scores": result.layer_scores,
        "gaps": result.gaps,
        "risk_assessment": result.risk_assessment,
        "recommendations": result.recommendations
    }

    with open("production_audit_results.json", "w") as f:
        json.dump(results_dict, f, indent=2)

    print("\n✅ Results saved to: production_audit_results.json")

print("\n" + "="*60)
print("🔗 NEXT STEPS")
print("="*60)
print()
print("Get production-ready code and daily resources:")
print("👉 https://community.nachiketh.in")
print()
print("Learn to build all 7 layers (production-grade):")
print("👉 https://bootcamp.nachiketh.in (Starts Feb 15)")
print()
print("="*60)
print()
print("Systems ship. Demos don't.")
print()


🎯 Production Readiness Audit for Agent Systems


This audit assesses your agent system across 7 layers:

Layer 1: Prompt Engineering (versioning, testing, templates)
Layer 2: Tool Selection (analytics, fallbacks, validation)
Layer 3: RAG & Memory (retrieval, chunking, overflow handling)
Layer 4: Agent Reasoning (tracing, loop detection, validation)
Layer 5: Orchestration (retry, idempotency, circuit breakers) ← CRITICAL
Layer 6: Observability (logging, metrics, tracing, alerts) ← CRITICAL
Layer 7: User Interface (versioning, errors, rate limiting, auth)

35 questions total. Takes 5-10 minutes.

Ready to start? Press Enter to begin...



🔍 PRODUCTION READINESS AUDIT

Answer each question honestly:
  YES (10 pts) - Fully implemented
  PARTIAL (5 pts) - Partially implemented
  NO (0 pts) - Not implemented


------------------------------------------------------------
LAYER 1: PROMPT ENGINEERING
------------------------------------------------------------

L1Q1: Do you have versioned promp

In [ ]:
#!/usr/bin/env python3
"""
Generate a polished HTML report from a production audit JSON file.

Usage:
  python3 generate_html_report.py /mnt/data/production_audit_results.json report.html
"""

import json
import math
import sys
from datetime import datetime

DEFAULT_INPUT = "/mnt/data/production_audit_results.json"
DEFAULT_OUTPUT = "production_audit_report.html"


def clamp(x, lo, hi):
    return max(lo, min(hi, x))


def esc(s: str) -> str:
    """Basic HTML escaping."""
    return (
        str(s)
        .replace("&", "&amp;")
        .replace("<", "&lt;")
        .replace(">", "&gt;")
        .replace('"', "&quot;")
        .replace("'", "&#39;")
    )


def score_tier(score_0_100: float):
    """
    Maps a 0-100 score to a tier + color.
    """
    s = clamp(score_0_100, 0, 100)
    if s >= 85:
        return ("Excellent", "#16a34a")
    if s >= 70:
        return ("Good", "#22c55e")
    if s >= 55:
        return ("Needs Work", "#f59e0b")
    if s >= 40:
        return ("High Risk", "#f97316")
    return ("Critical Risk", "#ef4444")


def severity_from_line(line: str):
    u = line.upper()
    if "URGENT" in u or "🚨" in line:
        return ("URGENT", "#ef4444")
    if "CRITICAL" in u:
        return ("CRITICAL", "#ef4444")
    if "WARNING" in u:
        return ("WARNING", "#f59e0b")
    if "INFO" in u:
        return ("INFO", "#3b82f6")
    return ("NOTE", "#64748b")


def ring_svg(percent: float, color: str):
    """
    Returns an inline SVG donut ring showing `percent` (0-100).
    """
    pct = clamp(percent, 0, 100)
    # Circle geometry
    r = 46
    cx = cy = 60
    circumference = 2 * math.pi * r
    dash = (pct / 100.0) * circumference
    gap = circumference - dash

    return f"""
    <svg width="120" height="120" viewBox="0 0 120 120" aria-label="Score ring">
      <defs>
        <filter id="shadow" x="-20%" y="-20%" width="140%" height="140%">
          <feDropShadow dx="0" dy="2" stdDeviation="3" flood-color="#000" flood-opacity="0.18"/>
        </filter>
      </defs>
      <circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="#e5e7eb" stroke-width="12"/>
      <circle cx="{cx}" cy="{cy}" r="{r}" fill="none"
              stroke="{color}" stroke-width="12"
              stroke-linecap="round"
              stroke-dasharray="{dash:.2f} {gap:.2f}"
              transform="rotate(-90 {cx} {cy})"
              filter="url(#shadow)"/>
      <text x="{cx}" y="{cy}" text-anchor="middle" dominant-baseline="middle"
            font-size="22" font-weight="800" fill="#0f172a">{pct:.0f}%</text>
      <text x="{cx}" y="{cy+22}" text-anchor="middle" dominant-baseline="middle"
            font-size="11" font-weight="600" fill="#64748b">Total</text>
    </svg>
    """


def bar_row(layer_label: str, score: float, max_score: float = 50.0):
    """
    A single horizontal bar row (layer score out of 50 by default).
    """
    ratio = 0 if max_score <= 0 else clamp(score / max_score, 0, 1)
    pct = ratio * 100
    # Color based on performance
    # 0-20 red, 20-35 orange, 35-45 amber, 45-50 green
    if score >= 45:
        color = "#22c55e"
    elif score >= 35:
        color = "#f59e0b"
    elif score >= 20:
        color = "#f97316"
    else:
        color = "#ef4444"

    return f"""
    <div class="bar-row">
      <div class="bar-left">
        <div class="layer-chip">Layer {esc(layer_label)}</div>
        <div class="bar-meta">{score:.0f}/50</div>
      </div>
      <div class="bar-track" role="img" aria-label="Layer {esc(layer_label)} score {score:.0f} out of 50">
        <div class="bar-fill" style="width:{pct:.1f}%; background:{color};"></div>
      </div>
    </div>
    """


def build_html(data: dict, source_path: str):
    total_score = float(data.get("total_score", 0.0))

    # Your JSON shows total_score like 34.285..., which looks like a % already.
    # We'll treat total_score as a 0-100 value.
    total_pct = clamp(total_score, 0, 100)
    tier_name, tier_color = score_tier(total_pct)

    layer_scores = data.get("layer_scores", {}) or {}
    # Sort layers numerically if possible
    try:
        layer_items = sorted(layer_scores.items(), key=lambda kv: int(str(kv[0])))
    except Exception:
        layer_items = sorted(layer_scores.items(), key=lambda kv: str(kv[0]))

    gaps = data.get("gaps", []) or []
    recommendations = data.get("recommendations", []) or []
    risk_assessment = data.get("risk_assessment", "")

    generated_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Build sections
    ring = ring_svg(total_pct, tier_color)

    bars_html = "\n".join(bar_row(str(k), float(v)) for k, v in layer_items)

    gaps_html_parts = []
    for g in gaps:
        sev, sev_color = severity_from_line(str(g))
        gaps_html_parts.append(
            f"""
            <div class="pill" style="border-left-color:{sev_color};">
              <div class="pill-top">
                <span class="sev" style="color:{sev_color};">{esc(sev)}</span>
              </div>
              <div class="pill-text">{esc(g)}</div>
            </div>
            """
        )
    gaps_html = "\n".join(gaps_html_parts) if gaps_html_parts else "<div class='muted'>No gaps listed.</div>"

    rec_lines = [r for r in recommendations if str(r).strip() != ""]
    rec_html = "\n".join(f"<li>{esc(line)}</li>" for line in rec_lines) if rec_lines else "<li class='muted'>No recommendations listed.</li>"

    # Risk callout
    risk_color = "#ef4444" if "CRITICAL" in str(risk_assessment).upper() else "#f59e0b"
    risk_html = f"""
    <div class="callout" style="border-left-color:{risk_color};">
      <div class="callout-title">Risk Assessment</div>
      <div class="callout-text">{esc(risk_assessment) if risk_assessment else "—"}</div>
    </div>
    """

    # HTML template (self-contained)
    html = f"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Production Audit Report</title>
  <style>
    :root {{
      --bg: #0b1020;
      --card: rgba(255,255,255,0.06);
      --card-2: rgba(255,255,255,0.08);
      --text: #e5e7eb;
      --muted: #94a3b8;
      --border: rgba(255,255,255,0.10);
      --shadow: 0 10px 30px rgba(0,0,0,0.35);
      --radius: 18px;
      --mono: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", "Courier New", monospace;
      --sans: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, "Apple Color Emoji", "Segoe UI Emoji";
    }}

    * {{ box-sizing: border-box; }}
    body {{
      margin: 0;
      font-family: var(--sans);
      color: var(--text);
      background:
        radial-gradient(1200px 600px at 20% 0%, rgba(59,130,246,0.30), transparent 60%),
        radial-gradient(900px 500px at 80% 10%, rgba(236,72,153,0.22), transparent 55%),
        radial-gradient(1000px 600px at 50% 100%, rgba(34,197,94,0.18), transparent 60%),
        var(--bg);
    }}

    .wrap {{
      max-width: 1080px;
      margin: 0 auto;
      padding: 28px 18px 46px;
    }}

    .header {{
      display: flex;
      align-items: flex-start;
      justify-content: space-between;
      gap: 16px;
      margin-bottom: 18px;
    }}

    .title {{
      font-size: 28px;
      font-weight: 900;
      letter-spacing: -0.02em;
      margin: 0;
      line-height: 1.15;
    }}

    .subtitle {{
      margin: 8px 0 0;
      color: var(--muted);
      font-size: 13px;
    }}

    .meta {{
      text-align: right;
      color: var(--muted);
      font-size: 12px;
      line-height: 1.4;
      white-space: nowrap;
    }}

    .grid {{
      display: grid;
      grid-template-columns: 1.1fr 0.9fr;
      gap: 16px;
      margin-top: 14px;
    }}

    @media (max-width: 900px) {{
      .grid {{ grid-template-columns: 1fr; }}
      .meta {{ text-align: left; white-space: normal; }}
    }}

    .card {{
      background: linear-gradient(180deg, var(--card-2), var(--card));
      border: 1px solid var(--border);
      border-radius: var(--radius);
      box-shadow: var(--shadow);
      padding: 18px 18px;
    }}

    .card h2 {{
      margin: 0 0 12px;
      font-size: 15px;
      letter-spacing: 0.02em;
      text-transform: uppercase;
      color: #cbd5e1;
    }}

    .score-row {{
      display: grid;
      grid-template-columns: 140px 1fr;
      gap: 14px;
      align-items: center;
    }}

    @media (max-width: 520px) {{
      .score-row {{ grid-template-columns: 1fr; }}
    }}

    .badge {{
      display: inline-flex;
      align-items: center;
      gap: 10px;
      padding: 10px 12px;
      border-radius: 999px;
      border: 1px solid var(--border);
      background: rgba(0,0,0,0.20);
      width: fit-content;
      font-weight: 700;
      margin-top: 8px;
    }}

    .dot {{
      width: 10px;
      height: 10px;
      border-radius: 999px;
      background: {tier_color};
      box-shadow: 0 0 0 4px rgba(255,255,255,0.06);
    }}

    .muted {{
      color: var(--muted);
    }}

    .bars {{
      margin-top: 10px;
      display: grid;
      gap: 10px;
    }}

    .bar-row {{
      display: grid;
      grid-template-columns: 170px 1fr;
      gap: 12px;
      align-items: center;
    }}

    @media (max-width: 520px) {{
      .bar-row {{ grid-template-columns: 1fr; }}
    }}

    .bar-left {{
      display: flex;
      align-items: center;
      justify-content: space-between;
      gap: 10px;
      min-width: 0;
    }}

    .layer-chip {{
      font-family: var(--mono);
      font-size: 12px;
      padding: 6px 10px;
      border-radius: 999px;
      border: 1px solid var(--border);
      background: rgba(0,0,0,0.20);
      white-space: nowrap;
    }}

    .bar-meta {{
      font-family: var(--mono);
      color: var(--muted);
      font-size: 12px;
      white-space: nowrap;
    }}

    .bar-track {{
      height: 12px;
      border-radius: 999px;
      background: rgba(255,255,255,0.08);
      border: 1px solid var(--border);
      overflow: hidden;
    }}

    .bar-fill {{
      height: 100%;
      border-radius: 999px;
      box-shadow: inset 0 -1px 0 rgba(0,0,0,0.25);
    }}

    .pill-grid {{
      display: grid;
      gap: 10px;
    }}

    .pill {{
      border: 1px solid var(--border);
      background: rgba(0,0,0,0.20);
      border-radius: 14px;
      padding: 12px 12px;
      border-left: 6px solid #64748b;
    }}

    .pill-top {{
      display: flex;
      align-items: center;
      justify-content: space-between;
      margin-bottom: 6px;
    }}

    .sev {{
      font-size: 12px;
      font-weight: 900;
      letter-spacing: 0.02em;
    }}

    .pill-text {{
      color: #e2e8f0;
      font-size: 13px;
      line-height: 1.35;
    }}

    .callout {{
      border: 1px solid var(--border);
      background: rgba(0,0,0,0.20);
      border-radius: 14px;
      padding: 12px 12px;
      border-left: 6px solid #f59e0b;
      margin-top: 12px;
    }}

    .callout-title {{
      font-size: 12px;
      font-weight: 900;
      text-transform: uppercase;
      letter-spacing: 0.02em;
      color: #cbd5e1;
      margin-bottom: 6px;
    }}

    .callout-text {{
      font-size: 13px;
      line-height: 1.35;
      color: #e2e8f0;
    }}

    ul {{
      margin: 0;
      padding-left: 18px;
      color: #e2e8f0;
      line-height: 1.45;
    }}

    li {{
      margin: 8px 0;
    }}

    .footer {{
      margin-top: 16px;
      color: var(--muted);
      font-size: 12px;
      display: flex;
      justify-content: space-between;
      gap: 12px;
      flex-wrap: wrap;
      opacity: 0.95;
    }}

    .mono {{
      font-family: var(--mono);
    }}
  </style>
</head>
<body>
  <div class="wrap">
    <div class="header">
      <div>
        <h1 class="title">Production Audit Report</h1>
        <div class="subtitle">
          Beautiful HTML summary generated from JSON audit output.
        </div>
      </div>
      <div class="meta">
        <div><span class="mono">Source:</span> {esc(source_path)}</div>
        <div><span class="mono">Generated:</span> {esc(generated_at)}</div>
      </div>
    </div>

    <div class="grid">
      <div class="card">
        <h2>Overall Score</h2>
        <div class="score-row">
          <div>{ring}</div>
          <div>
            <div style="font-size:18px; font-weight:900; margin-bottom:4px;">
              Tier: <span style="color:{tier_color};">{esc(tier_name)}</span>
            </div>
            <div class="muted" style="font-size:13px;">
              Total score interpreted as <span class="mono">{total_pct:.2f}</span> out of <span class="mono">100</span>.
            </div>
            <div class="badge">
              <span class="dot" aria-hidden="true"></span>
              <span>{esc(risk_assessment) if risk_assessment else "Risk assessment not provided"}</span>
            </div>
          </div>
        </div>

        {risk_html}
      </div>

      <div class="card">
        <h2>Layer Scores</h2>
        <div class="muted" style="margin-bottom:10px; font-size:13px;">
          Each layer is shown as a score out of <span class="mono">50</span>.
        </div>
        <div class="bars">
          {bars_html if bars_html.strip() else '<div class="muted">No layer scores provided.</div>'}
        </div>
      </div>
    </div>

    <div class="grid" style="margin-top:16px;">
      <div class="card">
        <h2>Gaps</h2>
        <div class="pill-grid">
          {gaps_html}
        </div>
      </div>

      <div class="card">
        <h2>Recommendations</h2>
        <ul>
          {rec_html}
        </ul>
      </div>
    </div>

    <div class="footer">
      <div>Report is a single HTML file (no external dependencies).</div>
      <div class="mono">Tip: share the .html as-is via email/Drive.</div>
    </div>
  </div>
</body>
</html>
"""
    return html


def main():
    in_path = "production_audit_results.json"
    out_path = "report.html"

    with open(in_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    html = build_html(data, in_path)

    with open(out_path, "w", encoding="utf-8") as f:
        f.write(html)

    print(f"✅ HTML report generated: {out_path}")

main()

✅ HTML report generated: report.html
